In [1]:
import pandas as pd

from datetime import datetime, timedelta

## Extract - Extração dos Dados

In [2]:
# Extração dos dados diretamente da API do Banco Central.

def extract_bacen(cod_serie, data_ini, data_fim):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{cod_serie}/dados?formato=json&dataInicial={data_ini}&dataFinal={data_fim}"
    df = pd.read_json(url)
    return df

Selic - 4189: Taxa de Juros - Selic acumulada no mês anuzalida - % a.a..

IPCA - 13522: Índice nacional de preços ao consumidor - amplo - 12 meses - %.

Câmbio (USD/BRL) - 3696: Taxa de Câmbio - Livre - Dólar Americano (venda) - Fim de período - mensal - u.m.c./US$.

In [3]:
data_ini = '01/01/2010'
data_fim = '31/07/2025'

selic = extract_bacen(4189, data_ini, data_fim)
ipca = extract_bacen(13522, data_ini, data_fim)
dolar = extract_bacen(3696, data_ini, data_fim)

## Transform - Tratamento dos Dados

In [4]:
# Tratamento dos dados, passando as datas como índice e dando nome a coluna de valores.

def transform(df, nome):
    df.set_index('data', inplace=True)
    df.index = pd.to_datetime(df.index, dayfirst=True)
    df.valor = df.valor.astype(float)
    df.rename(columns={'valor': nome}, inplace=True)
    return df

In [5]:
selic = transform(selic, 'selic')
ipca = transform(ipca, 'ipca')
dolar = transform(dolar, 'dolar')

In [6]:
# Merge dos dados em um único dataframe.

df = selic.merge(ipca, on='data').merge(dolar, on='data')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 187 entries, 2010-01-01 to 2025-07-01
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   selic   187 non-null    float64
 1   ipca    187 non-null    float64
 2   dolar   187 non-null    float64
dtypes: float64(3)
memory usage: 5.8 KB


In [8]:
df.isna().sum()

selic    0
ipca     0
dolar    0
dtype: int64

In [9]:
df.duplicated().sum()

0

## Load - Carregamento dos Dados

In [10]:
df.to_csv('dados_financeiros.csv', sep=';', encoding='utf-8')